In [1]:
import copy
import itertools as its
import math
import os
import pathlib
import re
import string
import sys
from typing import Dict, List, Optional, Tuple, Union
from collections import Counter, defaultdict, deque

import networkx as nx
import numpy as np
import pandas as pd
from IPython.display import clear_output
from matplotlib import pyplot as plt

from aoc import sim_new as sim, testing, util

twopi = 2 * math.pi

%matplotlib inline

INPUT_PATH = pathlib.Path('..') / 'input' / 'dec24.txt'

In [2]:
EMPTY = 0
FULL = 1

def read_data(text: str) -> List[List[int]]:
    data = []
    for line in text.rstrip().split('\n'):
        aline = []
        for c in line.rstrip():
            aline.append(EMPTY if c == '.' else FULL)
        data.append(aline)
    return data

def print_data(data: List[List[int]]):
    for line in data:
        for c in line:
            print('.' if c == EMPTY else '#', end='')
        print()


In [3]:
print_data(read_data(INPUT_PATH.read_text()))

.###.
##...
...##
.#.#.
#.#.#


In [4]:
def step(inp: List[List[int]]) -> List[List[int]]:
    out = [[EMPTY for _ in range(len(inp[0]))] for _ in range(len(inp))]
    for yy, line in enumerate(inp):
        for xx, c in enumerate(line):
            total_bugs_adjacent = 0
            for x, y in util.four_ways(xx, yy):
                if x < 0 or y < 0 or y >= len(inp) or x >= len(inp[y]):
                    continue
                if inp[y][x] == FULL:
                    total_bugs_adjacent += 1
            if c == EMPTY:
                if total_bugs_adjacent in (1, 2):
                    out[yy][xx] = FULL
            else:
                if total_bugs_adjacent == 1:
                    out[yy][xx] = FULL
    return out

In [5]:
test_input = """\
....#
#..#.
#..##
..#..
#...."""

test_output = """\
#..#.
####.
###.#
##.##
.##.."""

In [6]:
test_data = read_data(test_input)

In [7]:
print_data(step(test_data))

#..#.
####.
###.#
##.##
.##..


In [8]:
def hash_layout(data: List[List[int]]) -> int:
    output = 0
    # The reversed are to match the "biodiversity rating" of part 1
    for line in reversed(data):
        for c in reversed(line):
            output <<= 1
            output += 1 if c == FULL else 0
    return output

In [9]:
def step_until_dup(data: List[List[int]]):
    seen = set()

    while True:
        h = hash_layout(data)
        if h in seen:
            return data
        seen.add(h)
        data = step(data)

In [10]:
print(f'The answer to part 1 is {hash_layout(step_until_dup(read_data(INPUT_PATH.read_text())))}')

The answer to part 1 is 28717468


In [11]:
UP, DOWN, LEFT, RIGHT = 0, 1, 2, 3

def propose_down(inp, out):
    new_level = np.zeros_like(inp[0, :, :])
    last_level = inp[-1, :, :]
    new_level[:, 0] += last_level[2, 1]
    new_level[:, -1] += last_level[2, -2]
    new_level[0, :] += last_level[1, 2]
    new_level[-1, :] += last_level[-2, 2]
    new_level = (new_level > 0).astype(int)
    
    if (new_level == 1).any():
        return np.vstack([out, [new_level]])
    return out


def propose_up(inp, out):
    new_level = np.zeros_like(inp[0, :, :])
    last_level = inp[0, :, :]
    new_level[2, 1] = last_level[:, 0].sum()
    new_level[2, -2] = last_level[:, -1].sum()
    new_level[1, 2] = last_level[0, :].sum()
    new_level[-2, 2] = last_level[-1, :].sum()
    new_level = ((new_level == 1) | (new_level == 2)).astype(int)
    
    if (new_level == 1).any():
        return np.vstack([[new_level], out])
    return out


def sum_side(inp, side):
    if side == DOWN:
        return inp[:, 0].sum()
    if side == UP:
        return inp[:, -1].sum()
    if side == RIGHT:
        return inp[0, :].sum()
    if side == LEFT:
        return inp[-1, :].sum()
    raise ValueError('Bad side')


def recursive_step(inp):
    out = np.zeros_like(inp)
    size = inp.shape[1]
    for zz, face in enumerate(inp):
        for yy, line in enumerate(face):
            for xx, c in enumerate(line):
                if (xx, yy) == (2, 2):
                    # We just skip the recursive square
                    continue
                total_bugs_adjacent = 0
                for i, (x, y) in enumerate(util.four_ways(xx, yy)):
                    if x < 0 or y < 0 or y >= size or x >= size:
                        # Note only one of these conditions can be true at a time
                        # Try to recurse up
                        z = zz - 1
                        if z < 0:
                            # There's no space to recurse
                            continue
                        if x < 0:
                            x, y = 1, 2
                        elif y < 0:
                            x, y = 2, 1
                        elif x >= size:
                            x, y = -2, 2
                        else:
                            x, y = 2, -2
                        total_bugs_adjacent += inp[z][y][x]
                        
                    elif (x, y) == (2, 2):
                        # Try to recurse down
                        if zz == inp.shape[0] - 1:
                            # There are no bugs down below yet.
                            pass
                        else:
                            total_bugs_adjacent += sum_side(inp[zz + 1, :, :], i)
                    elif face[y][x] == FULL:
                        # We're just on the same level
                        total_bugs_adjacent += 1
                if c == EMPTY:
                    if total_bugs_adjacent in (1, 2):
                        out[zz][yy][xx] = FULL
                else:
                    if total_bugs_adjacent == 1:
                        out[zz][yy][xx] = FULL
    out = propose_down(inp, out)
    out = propose_up(inp, out)
    return out

In [12]:
data = np.array([read_data(INPUT_PATH.read_text())])
for _ in range(200):
    data = recursive_step(data)
print(f'The answer to part 2 is {data.sum()}')

The answer to part 2 is 2014
